In [ ]:
import pandas as pd

stadium_data = pd.read_csv('stadiums.csv')
# display(stadium_data)

nhl_stadium_data =  stadium_data[stadium_data['League'] == 'NHL']
display(nhl_stadium_data.head())
print(len(nhl_stadium_data))

# check what teams are in the nhl
nhl_teams = nhl_stadium_data['Team'].unique()
print(nhl_teams)

,Team,League,Division,Lat,Long
62,Tampa Bay Lightning,NHL,Atlantic,27.942700,-82.451800
63,Dallas Stars,NHL,Central,32.790556,-96.810278
64,Colorado Avalanche,NHL,Central,39.748700,-105.007600
65,Montreal Canadiens,NHL,Atlantic,45.496111,-73.569444
66,Nashville Predators,NHL,Central,36.159167,-86.778611


32
['Tampa Bay Lightning' 'Dallas Stars' 'Colorado Avalanche'
 'Montreal Canadiens' 'Nashville Predators' 'Winnipeg Jets'
 'Ottawa Senators' 'Washington Capitals' 'Seattle Kraken'
 'Los Angeles Kings' 'St. Louis Blues' 'Flordia Panthers'
 'Arizona Coyotes' 'Anaheim Ducks' 'Buffalo Sabres' 'Detroit Red Wings'
 'New York Rangers' 'Columbus Blue Jackets' 'Carolina Hurricanes'
 'Pittsburgh Penguins' 'New Jersey Devils' 'Vancouver Canucks'
 'Edmonton Oilers' 'San Jose Sharks' 'Toronto Maple Leafs'
 'Calgary Flames' 'Boston Bruins' 'Vegas Golden Knights'
 'New York Islanders' 'Chicago Blackhawks' 'Philadeliphia Flyers'
 'Minnesota Wild']


In [ ]:
# Arizona Coyotes moved to Utah as of 2024. Need to add this to the data.
# Challenge: if we are doing analysis for various years, need to track different teams for different years.

#TODO: get abbreviated name column for each team
#TODO: (nice to have) get the team logo for each team

## Proposed file structure

nhl-travel-analysis/<br>
├── extract-data.py <br>
├── clean-data.py<br>
├── analyze-data.py  # (Optional: Add if you plan to analyze the data)<br>
├── data/            # Directory for raw and cleaned data<br>
├── output/          # Directory for results or visualizations<br>
├── README.md        # Project description and instructions<br>
└── requirements.txt # Dependencies for the project<br>


### Let's try to get canucks schedule data and display as a df

In [4]:
import numpy as np
import pandas as pd
import requests
base_url = 'https://api-web.nhle.com/v1/'

# use requests to get nhl api data
def get_nhl_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # print(data)
        return 'Sucess!'
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")
    
def main():
    base_url = 'https://api-web.nhle.com/v1/'
    end_point = 'roster/VAN/current'
    test_url = base_url + end_point

    print(get_nhl_data(test_url))

if __name__ == '__main__':
    main()


Sucess!


In [8]:
def get_club_schedule(club, season):
    #/v1/club-schedule-season/{team}/{season}
    url = base_url + 'club-schedule-season/' + club + '/' + season
    print(url)
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")
    
# Example usage
canucks_schedule = get_club_schedule('VAN', '20242025')
print(canucks_schedule.keys())
print(canucks_schedule)

https://api-web.nhle.com/v1/club-schedule-season/VAN/20242025
dict_keys(['previousSeason', 'currentSeason', 'nextSeason', 'clubTimezone', 'clubUTCOffset', 'games'])
{'previousSeason': 20232024, 'currentSeason': 20242025, 'nextSeason': 20252026, 'clubTimezone': 'America/Vancouver', 'clubUTCOffset': '-07:00', 'games': [{'id': 2024010027, 'season': 20242025, 'gameType': 1, 'gameDate': '2024-09-24', 'venue': {'default': 'Rogers Arena'}, 'neutralSite': False, 'startTimeUTC': '2024-09-25T02:00:00Z', 'easternUTCOffset': '-04:00', 'venueUTCOffset': '-07:00', 'venueTimezone': 'America/Vancouver', 'gameState': 'FINAL', 'gameScheduleState': 'OK', 'tvBroadcasts': [{'id': 290, 'market': 'N', 'countryCode': 'CA', 'network': 'SNP', 'sequenceNumber': 33}, {'id': 324, 'market': 'N', 'countryCode': 'US', 'network': 'NHLN', 'sequenceNumber': 34}, {'id': 284, 'market': 'N', 'countryCode': 'CA', 'network': 'SN1', 'sequenceNumber': 102}, {'id': 554, 'market': 'A', 'countryCode': 'US', 'network': 'KHN', 'seq

The previousSeason, currentSeason, and nextSeason keys allow for pagination / accessing multiple years

In [ ]:
display(pd.DataFrame(canucks_schedule['games']))
# Filter for game type = 2 (regular season)
# TODO: get the winning team / losing team for each game

,id,season,gameType,gameDate,venue,neutralSite,startTimeUTC,easternUTCOffset,venueUTCOffset,venueTimezone,...,homeTeam,periodDescriptor,gameOutcome,winningGoalie,winningGoalScorer,threeMinRecap,gameCenterLink,condensedGame,threeMinRecapFr,condensedGameFr
0,2024010027,20242025,1,2024-09-24,{'default': 'Rogers Arena'},False,2024-09-25T02:00:00Z,-04:00,-07:00,America/Vancouver,...,"{'id': 23, 'commonName': {'default': 'Canucks'...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},"{'playerId': 8481668, 'firstInitial': {'defaul...","{'playerId': 8479425, 'firstInitial': {'defaul...",/video/recap-kraken-at-canucks-9-24-24-6362423...,/gamecenter/sea-vs-van/2024/09/24/2024010027,NaN,NaN,NaN
1,2024010034,20242025,1,2024-09-25,{'default': 'Abbotsford Centre'},True,2024-09-26T02:00:00Z,-04:00,-07:00,America/Los_Angeles,...,"{'id': 23, 'commonName': {'default': 'Canucks'...","{'periodType': 'OT', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'OT'},"{'playerId': 8480238, 'firstInitial': {'defaul...","{'playerId': 8478498, 'firstInitial': {'defaul...",NaN,/gamecenter/cgy-vs-van/2024/09/25/2024010034,NaN,NaN,NaN
2,2024010049,20242025,1,2024-09-27,{'default': 'Climate Pledge Arena'},False,2024-09-28T02:00:00Z,-04:00,-07:00,US/Pacific,...,"{'id': 55, 'commonName': {'default': 'Kraken'}...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},"{'playerId': 8478916, 'firstInitial': {'defaul...","{'playerId': 8477986, 'firstInitial': {'defaul...",/video/recap-canucks-at-kraken-9-27-24-6362584...,/gamecenter/van-vs-sea/2024/09/27/2024010049,NaN,NaN,NaN
3,2024010059,20242025,1,2024-09-28,{'default': 'Scotiabank Saddledome'},False,2024-09-29T01:00:00Z,-04:00,-06:00,US/Mountain,...,"{'id': 20, 'commonName': {'default': 'Flames'}...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},"{'playerId': 8478435, 'firstInitial': {'defaul...","{'playerId': 8476399, 'firstInitial': {'defaul...",/video/recap-canucks-at-flames-9-28-24-6362622...,/gamecenter/van-vs-cgy/2024/09/28/2024010059,NaN,NaN,NaN
4,2024010069,20242025,1,2024-09-30,{'default': 'Rogers Place'},False,2024-10-01T01:00:00Z,-04:00,-06:00,America/Edmonton,...,"{'id': 22, 'commonName': {'default': 'Oilers'}...","{'periodType': 'SO', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'SO'},"{'playerId': 8479973, 'firstInitial': {'defaul...",NaN,/video/recap-canucks-at-oilers-9-30-24-6362696...,/gamecenter/van-vs-edm/2024/09/30/2024010069,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2024021242,20242025,2,2025-04-08,{'default': 'American Airlines Center'},False,2025-04-09T00:00:00Z,-04:00,-05:00,US/Central,...,"{'id': 25, 'commonName': {'default': 'Stars'},...","{'periodType': 'OT', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'OT'},"{'playerId': 8477967, 'firstInitial': {'defaul...","{'playerId': 8480748, 'firstInitial': {'defaul...",/video/van-at-dal-recap-6371250121112,/gamecenter/van-vs-dal/2025/04/08/2024021242,/video/van-at-dal-condensed-game-6371249438112,/fr/video/van-vs-dal-08-04-2025-resume-6371249...,/fr/video/match-condense-canucks-stars-08-04-2...
84,2024021256,20242025,2,2025-04-10,{'default': 'Ball Arena'},False,2025-04-11T01:00:00Z,-04:00,-06:00,America/Denver,...,"{'id': 21, 'commonName': {'default': 'Avalanch...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},"{'playerId': 8480947, 'firstInitial': {'defaul...","{'playerId': 8480748, 'firstInitial': {'defaul...",/video/van-at-col-recap-6371341320112,/gamecenter/van-vs-col/2025/04/10/2024021256,/video/van-at-col-condensed-game-6371340838112,/fr/video/van-vs-col-10-04-2025-resume-6371340...,/fr/video/match-condense-canucks-avalanche-10-...
85,2024021273,20242025,2,2025-04-12,{'default': 'Rogers Arena'},False,2025-04-13T02:00:00Z,-04:00,-07:00,America/Vancouver,...,"{'id': 23, 'commonName': {'default': 'Canucks'...","{'periodType': 'OT', 'maxRegulationPeriods': 3}",{'lastPer